<a href="https://colab.research.google.com/github/darinakin/cQuant/blob/main/Darina_cQuant_coding_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [12]:
folder_path = '/Users/darina/Desktop/cQuant/hist'


### **Locational Nature of Power Prices**

**Task 1:** Read in all the historical data and combine it into a single data structure (data frame, table, etc.).

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
csv_files = glob.glob(folder_path + '*.csv')
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

In [16]:
file_names = ['ERCOT_DA_Prices_2016.csv', 'ERCOT_DA_Prices_2017.csv', 'ERCOT_DA_Prices_2018.csv', 'ERCOT_DA_Prices_2019.csv']


In [17]:
dfs = []


In [18]:
for file in file_names:
    df = pd.read_csv(file)
    dfs.append(df)

In [19]:
combined_df = pd.concat(dfs, ignore_index=True)


In [20]:
print(combined_df.head())


                  Date SettlementPoint  Price
0  2016-01-01 00:00:00       HB_BUSAVG  18.42
1  2016-01-01 00:00:00      HB_HOUSTON  18.42
2  2016-01-01 00:00:00       HB_HUBAVG  18.42
3  2016-01-01 00:00:00        HB_NORTH  18.43
4  2016-01-01 00:00:00        HB_SOUTH  18.38


In [21]:
combined_df.to_csv('combined_files.csv', index=False)


**Task 2:** Compute the average price for each settlement point and year-month in the historical dataset (48 year-months: January 2016 through December 2019).

In [22]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])


In [23]:
start_date = pd.Timestamp('2016-01-01')
end_date = pd.Timestamp('2019-12-31')
filtered_df = combined_df[(combined_df['Date'] >= start_date) & (combined_df['Date'] <= end_date)]


In [25]:
monthly_avg_prices = filtered_df.groupby([filtered_df['Date'].dt.to_period('M'), 'SettlementPoint'])['Price'].mean().reset_index()
print(monthly_avg_prices)



        Date SettlementPoint      Price
0    2016-01       HB_BUSAVG  19.209301
1    2016-01      HB_HOUSTON  19.565726
2    2016-01       HB_HUBAVG  19.258696
3    2016-01        HB_NORTH  19.133306
4    2016-01        HB_SOUTH  19.210054
..       ...             ...        ...
676  2019-12         LZ_LCRA  19.462621
677  2019-12        LZ_NORTH  18.999709
678  2019-12        LZ_RAYBN  19.079293
679  2019-12        LZ_SOUTH  19.798835
680  2019-12         LZ_WEST  39.765187

[681 rows x 3 columns]


**Task 3:** Write the computed monthly average prices to file as a CSV named AveragePriceByMonth**.csv.

In [26]:
output_file = 'AveragePriceByMonth.csv'
monthly_avg_prices.to_csv(output_file, index=False)

### **Price Volatility**

**Task 4:** Compute the hourly price1 volatility for each year and each settlement hub in the historical power price data (hubs are denoted by the prefix "HB_" in the SettlementPoint name).

In [27]:
filtered_df2 = filtered_df[(filtered_df['SettlementPoint'].str.startswith('HB_')) & (filtered_df['Price'] > 0)]


In [29]:
filtered_df2.loc[:, 'LogReturn'] = np.log(filtered_df2['Price'] / filtered_df2['Price'].shift(1))


In [40]:
filtered_df2.loc[:, 'Year'] = filtered_df2['Date'].dt.year


In [41]:
hourly_volatility = filtered_df2.groupby(['SettlementPoint', 'Year'])['LogReturn'].std().reset_index()
hourly_volatility.rename(columns={'LogReturn': 'HourlyVolatility'}, inplace=True)
hourly_volatility.to_csv('HourlyVolatilityByYear.csv', index=False)

**Task 6:** Determine which settlement hub showed the highest overall hourly volatility for each historical year. Write code to extract the rows of your computed hourly volatility table corresponding to these maxima and write this table to a second output file named MaxVolatilityByYear.csv.

In [42]:
max_volatility_by_year = hourly_volatility.loc[hourly_volatility.groupby('Year')['HourlyVolatility'].idxmax()]
max_volatility_by_year.to_csv('MaxVolatilityByYear.csv', index=False)

### **Data Translation and Formatting**

**Task 7:** Use the files in the supplementalMaterials folder as examples of the required data format, write code to translate the power price data structure you created in Task 1 into a format that could be readily consumed by the cQuant price simulation models, and write the data to separate files for each settlement point.

In [55]:
print(combined_df.columns)
print(combined_df.head())

Index(['Date', 'SettlementPoint', 'Price'], dtype='object')
         Date SettlementPoint  Price
0  2016-01-01       HB_BUSAVG  18.42
1  2016-01-01      HB_HOUSTON  18.42
2  2016-01-01       HB_HUBAVG  18.42
3  2016-01-01        HB_NORTH  18.43
4  2016-01-01        HB_SOUTH  18.38


In [57]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df['DateOnly'] = combined_df['Date'].dt.date
combined_df['Hour'] = combined_df['Date'].dt.hour + 1

In [58]:
pivoted_df = combined_df.pivot_table(index=['SettlementPoint', 'DateOnly'], columns='Hour', values='Price').reset_index()


In [60]:
import os


In [67]:
print(pivoted_df.columns)
print(len(pivoted_df.columns))


Index(['SettlementPoint', 'DateOnly', 1], dtype='object', name='Hour')
3


In [68]:
new_column_names = ['SettlementPoint', 'Date'] + [f'X{hour}' for hour in range(1, 25)]


In [69]:
if len(new_column_names) == len(pivoted_df.columns):
    pivoted_df.columns = new_column_names
else:
    print(f"Length mismatch: Expected {len(pivoted_df.columns)} elements, got {len(new_column_names)} elements.")

Length mismatch: Expected 3 elements, got 26 elements.


In [70]:
output_dir = 'formattedSpotHistory'
os.makedirs(output_dir, exist_ok=True)

In [71]:
for settlement_point in pivoted_df['SettlementPoint'].unique():
    sp_data = pivoted_df[pivoted_df['SettlementPoint'] == settlement_point]
    sp_data = sp_data.drop(columns=['SettlementPoint'])  # Drop the SettlementPoint column
    sp_data.to_csv(f'{output_dir}/spot_{settlement_point}.csv', index=False)